In [1]:
import altair as alt
from altair import datum
import pandas as pd
import numpy as np

In [2]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

### Obsolete

In [3]:
def make_gen_chart(csv_file):
    calls = pd.read_csv(csv_file)
    district_bool = ((calls['city_council_district'] == '2') |
                 (calls['city_council_district'] == '3') |
                 (calls['city_council_district'] == '4') |
                 (calls['city_council_district'] == '6') |
                 (calls['city_council_district'] == '7') |
                 (calls['city_council_district'] == '8'))
    dcalls = calls.loc[district_bool, :]
    
    type_bin = []
    
    pest_list = ['Pick up Dead Animal', 'Rodent Activity', 'Mice Infestation - Residential', 'Bed Bugs', 
                 'Pest Infestation - Residential', 'Pigeon Infestation']
    safety_list = ['Unsatisfactory Living Conditions', 'Unsafe Dangerous Conditions', 'Poor Conditions of Property', 
                   'Squalid Living Conditions', 'Chronic Dampness/Mold', 'Carbon Monoxide']
    utilities_list = ['Unsatisfactory Utilities - Electrical  Plumbing', 'No Utilities Residential - Gas', 
                      'No Utilities Residential - Electricity', 'No Utilities Residential - Water', 'Sewage/Septic Back-Up', 
                      'Water in Gas - High Priority']
    #district_dict = {'1': '1: Charlestown, East Boston, North End', '2':'2: Chinatown, Downtown, South Boston, South End',
    #                '3': '3: Dorchester', '4': '4: Mattapan, Dorchester, Roslindale, Jamaica Plain', 
    #                '5': '5: Hyde Park, Roslindale, Mattapan', '6': '6: Jamaica Plain, West Roxbury',
    #                '7': '7: Roxbury, South End, Dorchester', '8': '8: Back Bay, Beacon Hill, Fenway, Mission Hill, West End',
    #                '9': '9: Allston, Brighton'}

    for t in dcalls['type']:
        if t == 'Needle Pickup':
            type_bin.append(t)
        elif t in pest_list:
            type_bin.append('Dead Animals and Pests')
        elif t in safety_list:
            type_bin.append('Safety Concerns')
        elif t in utilities_list:
            type_bin.append('Lack of Utilities')
        else:
            type_bin.append('Other')
            
    dcalls['type_bin'] = type_bin
    
    year = csv_file.split('_')[1].split('.')[0]
    
    #dcalls.replace({'city_council_district': district_dict}, inplace=True)
    
    chart = alt.Chart(dcalls).mark_bar().encode(
            x=alt.X('type_bin:N', title='Type of Call'),
            y=alt.Y(
                'count()',
                title='Number of Calls'),
            facet=alt.Facet('city_council_district:O', columns=3, title='City Council District'),
            color = alt.Color('type_bin:N', title='Type of Call'),
            tooltip = 'count()'
        ).transform_filter(
            (datum.type_bin != 'Other')
        ).properties(title=f'311 Calls by District in {year}')
    
    chart = chart.configure_title(
            fontSize=20,
            font='Times New Roman',
            anchor='middle',
            color='black'
            )

    return chart

In [4]:
def make_spec_chart(csv_file, district):
    calls = pd.read_csv(csv_file)
    district_bool = (calls['city_council_district'] == district)
                     
    dcalls = calls.loc[district_bool, :]
    
    type_bin = []
    
    pest_list = ['Pick up Dead Animal', 'Rodent Activity', 'Mice Infestation - Residential', 'Bed Bugs', 
                 'Pest Infestation - Residential', 'Pigeon Infestation']
    safety_list = ['Unsatisfactory Living Conditions', 'Unsafe Dangerous Conditions', 'Poor Conditions of Property', 
                   'Squalid Living Conditions', 'Chronic Dampness/Mold', 'Carbon Monoxide']
    utilities_list = ['Unsatisfactory Utilities - Electrical  Plumbing', 'No Utilities Residential - Gas', 
                      'No Utilities Residential - Electricity', 'No Utilities Residential - Water', 'Sewage/Septic Back-Up', 
                      'Water in Gas - High Priority']
    trash_list = ['Requests for Street Cleaning', 'Illegal Dumping', 'Improper Storage of Trash (Barrels)', 
                 'Empty Litter Basket', 'Missed Trash/Recycling/Yard Waste/Bulk Item', 'Request for Recycling Cart',
                 'Request for Litter Basket Installation', 'Recycling Cart Return', 'Litter Basket Maintenance', 
                 'Overflowing or Un-kept Dumpster', 'Trash on Vacant Lot']

    for t in dcalls['type']:
        if t == 'Needle Pickup':
            type_bin.append(t)
        elif t in pest_list:
            type_bin.append('Dead Animals and Pests')
        elif t in safety_list:
            type_bin.append('Safety Concerns')
        elif t in utilities_list:
            type_bin.append('Lack of Utilities')
        elif t in trash_list:
            type_bin.append('Trash-Related')
        else:
            type_bin.append('Other')
            
    dcalls['type_bin'] = type_bin
    
    chart = alt.Chart(dcalls).mark_bar().encode(
                        x=alt.X('type:N', title='Type of Call', sort='-y'),
                        y=alt.Y('count()', title='Number of Calls'),
                        color=alt.Color('type_bin:N', title='Type of Call',
                                        scale=alt.Scale(
                                        domain=['Dead Animals and Pests', 'Lack of Utilities', 
                                                'Needle Pickup', 'Safety Concerns', 'Trash-Related', 'Other'],
                                        range=['#4c78a8', '#f58518', '#e45756', '#72b7b2', '#54a24b', 'gray'])
                                       ), tooltip='count()')
    
    return chart

### Usable

In [5]:
# Function takes the 311 dataset and makes a new df containing each individual type of report and how often it occurs
def trim_df(df):
    new_df = pd.DataFrame()
    
    df = df[['type', 'city_council_district']]
    
    dis_list = ['2', '3', '4', '6', '7', '8']
    
    for dis in dis_list:
        dis_bool = df['city_council_district'] == dis
        dis_df = df.loc[dis_bool]
        type_ser = dis_df.value_counts('type')
        block_df = pd.DataFrame(type_ser).reset_index()
        block_df.columns=['type', 'count']
        block_df['district'] = dis
        
        new_df = pd.concat([new_df, block_df])
        
    return new_df.reset_index()

In [6]:
# Function adds a new column to the trimmed dataframe denoting which category the calls fall into
def bin_df(df):
    type_bin = []
    
    pest_list = ['Pick up Dead Animal', 'Rodent Activity', 'Mice Infestation - Residential', 'Bed Bugs', 
                 'Pest Infestation - Residential', 'Pigeon Infestation']
    safety_list = ['Unsatisfactory Living Conditions', 'Unsafe Dangerous Conditions', 'Poor Conditions of Property', 
                   'Squalid Living Conditions', 'Chronic Dampness/Mold', 'Carbon Monoxide']
    utilities_list = ['Unsatisfactory Utilities - Electrical  Plumbing', 'No Utilities Residential - Gas', 
                      'No Utilities Residential - Electricity', 'No Utilities Residential - Water', 'Sewage/Septic Back-Up', 
                      'Water in Gas - High Priority']
    
    for t in df['type']:
        if t == 'Needle Pickup':
            type_bin.append(t)
        elif t in pest_list:
            type_bin.append('Dead Animals and Pests')
        elif t in safety_list:
            type_bin.append('Safety Concerns')
        elif t in utilities_list:
            type_bin.append('Lack of Utilities')
        else:
            type_bin.append('Other')
            
    df['type_bin'] = type_bin
    
    return df

In [7]:
# Function creates a new dataframe that counts the instances of each category of call
def sum_df(df):
    
    new_df = pd.DataFrame()
    
    for dis in df['district'].unique():
        
        block_df = pd.DataFrame()
        
        dis_bool = df['district'] == dis
        dis_df = df.loc[dis_bool]
        
        count_list = []
        type_list = []
        
        for t in dis_df['type_bin'].unique():
            t_bool = dis_df['type_bin'] == t
            t_df = dis_df.loc[t_bool]
            total = t_df['count'].sum()
            
            count_list.append(total)
            type_list.append(t)
        
        block_df['type_bin'] = type_list
        block_df['count'] = count_list
        block_df['district'] = dis
        
        new_df = pd.concat([new_df, block_df])
        
    return new_df

In [8]:
calls_2022 = pd.read_csv('311_2022.csv')
calls_2022 = sum_df(bin_df(trim_df(calls_2022)))

In [9]:
calls_2022

,type_bin,count,district
0,Other,50526,2
1,Needle Pickup,2382,2
2,Dead Animals and Pests,1989,2
3,Safety Concerns,1266,2
4,Lack of Utilities,56,2
0,Other,22696,3
1,Needle Pickup,1075,3
2,Safety Concerns,1148,3
3,Dead Animals and Pests,1586,3
4,Lack of Utilities,48,3


In [16]:
chart_2022 = alt.Chart(calls_2022).mark_bar().encode(
            x=alt.X('type_bin:N', title='Type of Call'),
            y=alt.Y('count:Q',
                title='Number of Calls'),
            facet=alt.Facet('district:O', columns=3, title='City Council District'),
            color = alt.Color('type_bin:N', title='Type of Call'),
            tooltip = 'count:Q'
        ).transform_filter(
            (datum.type_bin != 'Other')
        ).properties(title=f'311 Calls by District in 2022')
    
chart_2022 = chart_2022.configure_title(
        fontSize=20,
        font='Times New Roman',
        anchor='middle',
        color='black'
        )
chart.save("311_2022.html")
chart_2022

alt.Chart(...)